In [ ]:
import requests
import json
import time
from bs4 import BeautifulSoup

In [ ]:
base_url = "https://kurser.dtu.dk/course/"
cookies_base = {
    'ASP.NET_SessionId' : "m2nar0ff40qtrxgahijrrek0",
    '{DTUCoursesPublicLanguage}' : 'en-GB'
}
cookies_eval = {
    'ASP.NET_SessionId' : "jyl2tu1kp5rftikcrflzznsu",
    '{DTUCoursesPublicLanguage}' : 'en-GB',
}
cookies_grades = {
    'ASP.NET_SessionId' : "anw5qpxstahbxysv200oygx5",
    '{DTUCoursesPublicLanguage}' : 'en-GB'
}

In [ ]:
with open('valid_courses.json') as f:
    valid_courses = json.load(f)

In [ ]:
eval_links = {}

for course_num in valid_courses:
    
    # Define page content
    course_url = f"{base_url}{course_num}/info"
    response = requests.get(course_url, cookies=cookies_base)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find left div, and all links within it
    left_box = soup.find('div', class_='box')
    atags = left_box.find_all('a')
        
    # Extract link-urls and link-texts
    eval_hrefs = []
    eval_texts = []
    grade_hrefs = []
    grade_texts = []
    for atag in atags:
        if "evaluering.dtu.dk" in atag['href']:
            eval_hrefs.append(atag['href'])
            eval_texts.append(atag.text.replace('\r', '').replace('\n', '').replace('  ', ''))
        elif "karakterer.dtu.dk" in atag['href']:
            grade_hrefs.append(atag['href'])
            grade_texts.append(atag.text.replace('\r', '').replace('\n', '').replace('  ', ''))

    if (len(eval_hrefs) == 0) or (len(grade_hrefs) == 0):
        print(f"Not enough info on {course_num}")
        continue
    
    # Get content of the links
    print(course_url)
    eval_response = requests.get(eval_hrefs[0], cookies=cookies_eval)
    grade_response1 = requests.get(grade_hrefs[0], cookies=cookies_grades)
    if len(grade_hrefs) > 1:
        grade_response2 = requests.get(grade_hrefs[1], cookies=cookies_grades)
        
        # Second latest grade
        grade2_soup = BeautifulSoup(grade_response2.content, 'html.parser')
        form_2 = grade2_soup.find('form', id='karsumForm')
        num_participants_2 = int(form_2.find_all('td')[1].text.replace('\r', '').replace('\n', '').replace('  ', ''))
    else:
        num_participants_2 = 0
    
    # Latest grade
    grade1_soup = BeautifulSoup(grade_response1.content, 'html.parser')
    form_1 = grade1_soup.find('form', id='karsumForm')
    num_participants_1 = int(form_1.find_all('td')[1].text.replace('\r', '').replace('\n', '').replace('  ', ''))
    
    
    
    # Based on the number of participants, choose the correct grade page
    if num_participants_1*2 > num_participants_2:
        eval_links[course_num] = {'eval': eval_hrefs[0], 'grade': grade_hrefs[0]}
    else:
        eval_links[course_num] = {'eval': eval_hrefs[0], 'grade': grade_hrefs[1]}   
    

In [ ]:
eval_links

In [ ]:
# save dict as json
with open('eval_links.json', 'w') as file:
    json.dump(eval_links, file)